# The underlying mathematics of Gillespie simulation

Gillespie simulation is an event-by-event method of simulating systems. Its essence is the observation that we can manipulate the probabilities governing events. Instead of testing in every discrete timestep which of the available events can occur (for example from susceptible to infected in SIR), we predict the instant of time at which the next event will occur &ndash; skipping the intermediate time when nothing happens.

To put this another way, we convert the probabilities of individual events in *space* into aggregate probability distributions of events over *time*. If the simulation is such that a lot of "empty" timesteps occur, then this approach will avoid the costs of simulating them. It has the additional advantage of operating in continuous time with only a single event happening at each instant, which solves the problem of events affecting each other within a timestep. 

Unfortunately these benefits come at the cost of some fairly subtle mathematics needed to manipulate the probability distributions into the required form. We'll deal with this first, and then encode the result as a new simulation dynamics that we can use to simulate epidemics using the *same* compartmented process models as we used for the synchronous case.

## From single-step to interval behaviour

In synchronous simulation we took all the opportunities for an event could occur &ndash; all the places at which an event could fire &ndash;  and probabilistically chose some of them for firing. Using the SIR model as an example, infections happen along SI edges (the edges with a susceptible node at one end and an infected node at the other) independently. If we denote the probability of an SI edge transmitting an infection as $p_{infect}$, then the rate at which edges in the network transmit infection is given by $p_{infect} \, N_{infect}$ where $N_{infect}$ denotes the number of SI edges in the network (the number of infection opportunities). $N _{infect}$ is of course a function of time, since the population of SI edges is changed by the infection event. 
Similarly if infected nodes are removed with probability $p_{remove}$ the rate of recovery is given by $p_{remove} \, N_{remove}$.

Each infection event will decrease $N_{infect}$ by one (the SI edge that the disease passed over, and which is now an II edge); increase $N_{remove}$ by one (since there is now another opportunity for a removal event fired agaionst the newly-infected node); and increase $N_{infect}$  by the number of SI edges formed by having susceptible nodes adjacent to the newly-infected node. This indicates that the dynamics entwines three distinct features:

1. the probabilities of different events;
1. the number of places at which these events can occur; and
1. the topology of the network that controls how the populations of different nodes and edges evolve.

### The joint event probability distribution

Let us re-formulate the above in a way that's more explcitly continuous in nature. The probability that some SI edge will transmit infection in a small time $dt$ is given by $a_{infect} \, dt = p_{infect} \, N_{infect} \, dt$, and recovery similarly by $a_{remove} \, dt = p_{remove} \, N_{remove} \, dt$, where $a_{infect}$ denotes the rate of infection events at a given time. We can now ask two questions: given the state of the network,

1. when will the next event occur?, and
1. what event will it be?

Clearly these are probabilistic questions, so the answers will be formulated as probability distributions. Let's define a probability distribution $P(\tau, e) \, d\tau$ as the probability that an event will happen in the interval $(t + \tau, t + \tau + d\tau)$ *and* that that event will be of type $e$, which for SIR will be either an infection or a removal event. So at time $t$ we're looking at the distribution of the times $\tau$ between $t$ and the next event, and the identity of that event. This is a joint probability density function on the space of $\tau$ and $e$, where $\tau$ is a continuous random variable and $e$ is a discrete random variable. We an then draw values a pair of values  $(\tau, e)$ from this distribution to give us the time to the next event and its identity. 

Note also that the value of $\tau$ answers the first question above, while the value of $e$ answers the second.

What do we expect from this distribution? Intuitively, a system where there are lots of places where events can occur should give rise to a high likelihood of drawing a small value of $\tau$ from the distribution: the events happen close together in time. Conversely, as the number of places available decreases, it becomes more likely that we'll draw a larger value of $\tau$.

### The distribution of events in time

We now need a way to specify $P(\tau, e)$ and to draw values from it.

Let's think about $P(\tau, e) \, d\tau$ a little more. We're looking for a value of $\tau$ at which the next event happens, and the identity of that event. Equivalently, we could say that we want the probability that *no* event happens in the interval $[t, t + \tau]$, *and* that an $e$ event happens in the interval $[t + \tau, t + \tau + d\tau]$. The use of the word "and" here suggests that we'll be multiplying together the probabilities of the two components. We defined to probability of a particular event happening above, so we can then re-phrase $P(\tau, e) \, d\tau$ a little differently

$$
    P(\tau, e) \, d\tau = P_0(\tau) \, a_e \, d\tau
$$

where $P_0(\tau)$ is the probability of no event happening in $(t + \tau)$ and $a_e$ is the probability of *some* event $e$ happening in an interval $d\tau$. Since we already know the values of $a_e$ from the model parameters $\alpha$ and $\beta$ and the size of the appropriate loci $[SI]$ and $[I]$, we just need an expression for $P_0(\tau)$. Let $a \, d\tau' = \sum_e a_e \, d\tau'$ be the probability that *some* event happens in an interval $d\tau'$, simply by summing-up the component probabilities of the different events. We then have

$$
    P_0(\tau' + d\tau') = P_0(\tau') \, (a \, d\tau')
$$

which is the probability that no event occurs in in the interval $(t, t + \tau)$ *and then* that one occured in the following interval $d\tau'$. This is a differential equation, the solution of which is

$$
    P_0(\tau) = e^{-a \tau}
$$

Substituting back into the above we therefore have

$$
    P(\tau, e) = P_0(\tau) \, a_e
               = a_e \, e^{-a \tau}
$$

This is our joint probability distribution for the events defined by the various values of $a_e$. These values are *rates*, not probabilities: they are defined in terms of the number of places at which each event $e$ can occur. 

### Drawing from the distribution

To conduct simulation, we need to be able to draw a pair $(\tau, e)$ from our distribution. However, we can't simply choose $\tau$ and $e$ independently of each other, because the value of $P(\tau, e)$ depends on *all* the possible events $e$ because of the presence of $a$, the sum of all event rates, in its definition. That means that the time to the next event depends on the number of events that could occur.

In other words, $P(\tau, e)$ is a **joint probability distribution** from which we need to draw a pair. Any joint probability distribution $P(a, b)$ can be re-written as $P(a, b) = P(a) \, P(b | a)$: the prior (independent) probability of $a$ occuring multiplied by the probability of $b$ occurring *given that* $a$ has occurred. In our case

$$
    P(\tau, e) = P(\tau) \, P(e | \tau)
$$

where $P(\tau)$ is the probability that *some* event will occur on the interval $(t, t + \tau)$ and $P(e | \tau)$ is the probability that this event will be of type $e$ *given that* it occurs on this interval. Clearly $P(\tau)$ is simply the sum of the probabilities for all the events that may occur

$$
    P(\tau) = \sum_e P(\tau, e)
$$

and therefore

$$
    P(e | \tau) = \frac{P(\tau, e)}{\sum_{e'} P(\tau, e')}
$$

These two equations are both single-variable probability distributions (over $\tau$ and $e$ respectively) expressed in terms of the joint probability distribution $P(\tau, e)$, and if we substitute for $P(\tau, e)$ from above we get

$$
\begin{align*}
    P(\tau)     &= \sum_e a_e e^{-a \tau} \\
                &= a e^{-a \tau} \\
    P(e | \tau) &= \frac{P(\tau, e)}{\sum_{e'} P(\tau, e')} \\
                &= \frac{a_e e^{-a \tau}}{a e^{-a \tau} } \\
                &= \frac{a_e}{a}
\end{align*}
$$

Note that $P(e | \tau)$ is in this case independent of $\tau$, since the event probabilities are constants.

Let's briefly return to the network scenario we're interested in. The value $\tau$ is the interval of time until the next event occurs in the network, whether that is the infection of the S node attached to an SI edge of the recovery of an I node. Which of these events happens is determined by $e$. The pair $(\tau, e)$ therefore fully defines the time and identity of the next event in the simulation. It remains to see how we choose these two values, and how the network evolves in response to the selected event.

### <span id="sec_epidemic_gillespie_drawing_pair"></span> Generating the $(\tau, e)$ pair

In order to make use of $P(\tau, e)$ we have to be able to draw $\tau$ and $e$ from the joint distribution. We saw above that we can dop this by drawing values from $P(\tau)$ and $P(e | \tau)$ individually, with the latter distribution actually being independent of time in our current case.

It may not be obvious how to draw from such distributions, but we can manipulate the probabilities to make it possible using only a source of uniformly-distributed random numbers on the range $(0, 1)$. The trick is to observe that, for any probability density function $P(a)$, the value $P(a) \, da$ represents the probability that a value drawn from the distribution will lie between $a$ and $(a + da)$. From this we can construct a cumulative distribution function,

$$
    F(x) = \int_{-\infty}^{x} P(a) \, da
$$

where $F(x_0)$ represents the probability that a value drawn from $P(a)$ is less than or equal to $x_0$, also denoted $P(a \le x_0)$. If we now draw a value $r$ from a uniform distribution on $(0, 1)$ we can compute $x = F^{-1}(r)$ where $F^{-1}$ is the inverse of the cumulative distribution function and $x$ will be distributed according to $P(a)$.  This means we can convert a uniformly-distributed value into a value drawn from any probability distribution for which we can construct (and invert) a cumulative distribution function.

In our case we have that $P(\tau) = a \, e^{-a \tau}$. Remember that $a$ is a constant, and that intervals can't be negative. This means that

\begin{align*}
    F(\tau) &= \int_{-\infty}^{\tau} a \, e^{-a \tau'} \, d\tau' \\
            &= \int_0^{\tau} a \, e^{-a \tau'} \, d\tau' \\
            &= -e^{-a \tau'} \, \bigg|_0^\tau \\
            &= -e^{-a \tau} -(-e^0) \\
            &= 1 - e^{-a \tau}
\end{align*}

This is an awkward expression to manipulate, but we can observe that, if a number $r_1$ is uniformly distributed, then so by definition is $1 - r_1$, so if we set $F(\tau) = 1 - r_1$ we can cancel-out the constant ones and get a simpler expression overall. We then have

\begin{align*}
      1 - r_1 &= F(\tau) \\
              &= 1 - e^{-a \tau} \\
          r_1 &= e^{-a \tau} \\
              &= \frac{1}{e^{a \tau}} \\
   e^{a \tau} &= \frac{1}{r_1} \\
       a \tau &= \ln \frac{1}{r_1} \\
         \tau &= \frac{1}{a} \, \ln \frac{1}{r_1}
\end{align*}

The discrete case works similarly. If we draw a value $r_2$ on $(0, 1)$, then the value of $e$ we require is given by $\sum_{e' = 0}^{e - 1} a_{e'} \leq r_2 a \leq \sum_{e' = 0}^{e} a_{e'}$: the largest $e$ such that the sum of $a_{e'}$ for $e' \le e$ is less than $r_2 a$.

The upshot of all this probability theory is that we can choose a time to the next event $\tau$ and the identity of the next event $e$ from the distribution induced by the individual event probabilities and the size of the loci for the various events in the network, by drawing two uniformly-distributed numbers and performing two simple calculations.  